In [2]:
# 1. MongoClient 임포트
from pymongo import MongoClient

import pymongo
print(pymongo.__version__)

4.13.0


In [3]:
# 2. MongoClient 객체 생성
# MongoDB는 기본적으로 27017 포트를 사용합니다.
# client = MongoClient('localhost', 27017) 와 동일합니다.
client = MongoClient('mongodb://localhost:27017/')

In [4]:
# 3. 데이터베이스 선택
# 만약 'my_database'라는 데이터베이스가 없으면,
# 데이터를 처음 쓸 때 자동으로 생성됩니다.
db = client['my_database']

In [5]:
# 4. 컬렉션 선택
# 데이터베이스 선택과 마찬가지로, 'my_collection'이 없으면
# 데이터를 처음 쓸 때 자동으로 생성됩니다.
collection = db['my_collection']

In [6]:
print("MongoDB 연결 성공!")
print("선택된 DB:", db.name)
print("선택된 Collection:", collection.name)

# 연결 확인 후 클라이언트 닫기 (실제 애플리케이션에서는 계속 열어둡니다)
client.close()

MongoDB 연결 성공!
선택된 DB: my_database
선택된 Collection: my_collection


In [8]:
client = MongoClient('mongodb://localhost:27017/')
db = client['my_database']
collection = db['my_collection']

# 삽입할 문서(Dictionary) 생성
user_doc = {
    "name": "백인호",
    "email": "ino1533@gmail.com",
    "age": 30,
    "skills": ["자바", "오라클", "스프링부트개발"]
}

# insert_one() 메소드로 문서 삽입
result = collection.insert_one(user_doc)

# 삽입된 문서의 고유 ID(_id) 확인
print(f"하나의 문서가 삽입되었습니다. ID: {result.inserted_id}")

client.close()


하나의 문서가 삽입되었습니다. ID: 68479db28f5735a55d32acbd


In [9]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['my_database']
collection = db['my_collection']

# 삽입할 문서 리스트 생성
user_docs = [
    {"name": "이순신", "email": "sunsin@example.com", "age": 45},
    {"name": "유관순", "email": "gwansun@example.com", "age": 17},
    {"name": "세종대왕", "email": "sejong@example.com", "age": 32}
]

# insert_many() 메소드로 여러 문서 삽입
result = collection.insert_many(user_docs)

# 삽입된 문서들의 고유 ID 리스트 확인
print(f"여러 문서가 삽입되었습니다. IDs: {result.inserted_ids}")

client.close()


여러 문서가 삽입되었습니다. IDs: [ObjectId('68479f5c8f5735a55d32acbf'), ObjectId('68479f5c8f5735a55d32acc0'), ObjectId('68479f5c8f5735a55d32acc1')]


In [10]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['my_database']
# 이전 데이터를 깔끔하게 지우고 새로 시작하고 싶다면 아래 라인의 주석을 해제하세요.
# db.users.drop()
users_collection = db['users']

# 데이터가 비어있을 때만 샘플 데이터 삽입
if users_collection.count_documents({}) == 0:
    users_collection.insert_many([
        {"name": "홍길동", "age": 25, "city": "서울", "skills": ["Python", "DB"]},
        {"name": "이순신", "age": 45, "city": "서울", "skills": ["Leadership", "Strategy"]},
        {"name": "유관순", "age": 17, "city": "충남", "skills": ["History", "Public Speaking"]},
        {"name": "세종대왕", "age": 32, "city": "서울", "skills": ["Python", "AI", "Korean"]},
        {"name": "장영실", "age": 28, "city": "부산", "skills": ["Python", "Engineering"]}
    ])
    print("샘플 데이터가 삽입되었습니다.")


샘플 데이터가 삽입되었습니다.


In [12]:
# 조건 없이 첫 번째 문서 하나 가져오기
first_user = users_collection.find_one()
print(first_user)

# 조건에 맞는 문서 하나 가져오기
sejong = users_collection.find_one({"name": "세종대왕"})
print(sejong)


{'_id': ObjectId('6847a0908f5735a55d32acc3'), 'name': '홍길동', 'age': 25, 'city': '서울', 'skills': ['Python', 'DB']}
{'_id': ObjectId('6847a0908f5735a55d32acc6'), 'name': '세종대왕', 'age': 32, 'city': '서울', 'skills': ['Python', 'AI', 'Korean']}


In [11]:
# '서울'에 사는 모든 사용자 조회
seoul_users = users_collection.find({"city": "서울"})

print(type(seoul_users)) # <class 'pymongo.cursor.Cursor'>

for user in seoul_users:
    print(user)


<class 'pymongo.synchronous.cursor.Cursor'>
{'_id': ObjectId('6847a0908f5735a55d32acc3'), 'name': '홍길동', 'age': 25, 'city': '서울', 'skills': ['Python', 'DB']}
{'_id': ObjectId('6847a0908f5735a55d32acc4'), 'name': '이순신', 'age': 45, 'city': '서울', 'skills': ['Leadership', 'Strategy']}
{'_id': ObjectId('6847a0908f5735a55d32acc6'), 'name': '세종대왕', 'age': 32, 'city': '서울', 'skills': ['Python', 'AI', 'Korean']}


In [15]:
# 나이가 30세 미만인 사용자 조회
young_users = users_collection.find({"age": {"$lt": 20}})
for user in young_users:
    print(user['name'], user['age'])


유관순 17


In [19]:
# '홍길동' 또는 '장영실' 조회
some_users = users_collection.find({"name": {"$in": ["홍길동", "장영실"]}})
for user in some_users:
    print(user['name'])

# 'Python' 스킬을 보유한 모든 사용자 조회 (배열 필드 쿼리)
python_users = users_collection.find({"skills": "Python"}) # 이 방식도 가능!
for user in python_users:
    print(f"{user['name']}님은 Python 능력자!")


홍길동
장영실
홍길동님은 Python 능력자!
세종대왕님은 Python 능력자!
장영실님은 Python 능력자!


In [22]:
# AND 조건: 서울에 살면서, 나이가 30세 이상인 사용자
# 아래 두 쿼리는 동일하게 동작합니다.
# 1. 암시적 AND
query1 = {"city": "서울", "age": {"$gte": 30}}
# 2. 명시적 AND (같은 필드에 다른 조건을 걸 때 유용)
query2 = {"$and": [{"city": "서울"}, {"age": {"$gte": 30}}]}

users = users_collection.find(query1)
for user in users:
    print(f"[AND] {user['name']}")

# OR 조건: 부산에 살거나, 'Leadership' 스킬을 가진 사용자
query3 = {"$or": [{"city": "부산"}, {"skills": "Leadership"}]}
users = users_collection.find(query3)
for user in users:
    print(f"[OR] {user['name']} {user['city']} {user['skills']}")


[AND] 이순신
[AND] 세종대왕
[OR] 이순신 서울 ['Leadership', 'Strategy']
[OR] 장영실 부산 ['Python', 'Engineering']


In [23]:
query4 = {"$nor": [{"city": "부산"}, {"skills": "Leadership"}]}
users = users_collection.find(query4)
for user in users:
    print(f"[OR] {user['name']} {user['city']} {user['skills']}")

[OR] 홍길동 서울 ['Python', 'DB']
[OR] 유관순 충남 ['History', 'Public Speaking']
[OR] 세종대왕 서울 ['Python', 'AI', 'Korean']


In [24]:
# 모든 사용자의 이름과 나이만 가져오기 (_id는 자동으로 포함됨)
results = users_collection.find({}, {"name": 1, "age": 1})
for r in results:
    print(r)

# 모든 사용자의 이름과 나이만 가져오기 (_id는 제외)
results = users_collection.find({}, {"name": 1, "age": 1, "_id": 0})
for r in results:
    print(r)


{'_id': ObjectId('6847a0908f5735a55d32acc3'), 'name': '홍길동', 'age': 25}
{'_id': ObjectId('6847a0908f5735a55d32acc4'), 'name': '이순신', 'age': 45}
{'_id': ObjectId('6847a0908f5735a55d32acc5'), 'name': '유관순', 'age': 17}
{'_id': ObjectId('6847a0908f5735a55d32acc6'), 'name': '세종대왕', 'age': 32}
{'_id': ObjectId('6847a0908f5735a55d32acc7'), 'name': '장영실', 'age': 28}
{'name': '홍길동', 'age': 25}
{'name': '이순신', 'age': 45}
{'name': '유관순', 'age': 17}
{'name': '세종대왕', 'age': 32}
{'name': '장영실', 'age': 28}


In [25]:
# 나이순으로 내림차순 정렬하여 상위 3명만 조회
top3_users = users_collection.find().sort("age", -1).limit(3)
print("--- 나이 많은 순 Top 3 ---")
for user in top3_users:
    print(f"{user['name']} ({user['age']})")

# 페이지네이션 예제: 한 페이지에 2명씩, 2번째 페이지 조회
# 1페이지: skip(0).limit(2)
# 2페이지: skip(2).limit(2)
page_size = 2
page_num = 2
paged_users = users_collection.find().sort("name", 1).skip((page_num - 1) * page_size).limit(page_size)
print(f"--- 이름순 정렬 {page_num} 페이지 ---")
for user in paged_users:
    print(user['name'])


--- 나이 많은 순 Top 3 ---
이순신 (45)
세종대왕 (32)
장영실 (28)
--- 이름순 정렬 2 페이지 ---
이순신
장영실


In [27]:
# [update_one] 홍길동의 나이를 26으로 수정($set)
result = users_collection.update_one(
    {"name": "홍길동"},
    {"$set": {"age": 26}}
)
print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")

# [update_one] 홍길동의 나이를 1 증가($inc)하고, 사는 곳 필드 추가
result = users_collection.update_one(
    {"name": "홍길동"},
    {
        "$inc": {"age": 1},
        "$set": {"location": "Seoul, KR"}
    }
)
# [unset] 홍길동의 location 필드 제거
users_collection.update_one({"name": "홍길동"}, {"$unset": {"location": 1}})


Matched: 1, Modified: 0
Matched: 3, Modified: 3


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [28]:
users_collection.find_one({"name":"이순신"})

{'_id': ObjectId('6847a0908f5735a55d32acc4'),
 'name': '이순신',
 'age': 45,
 'city': '서울특별시',
 'skills': ['Leadership', 'Strategy']}

In [29]:
# [update_many] '서울'에 사는 모든 사용자의 city를 '서울특별시'로 변경
result = users_collection.update_many(
    {"city": "서울"},
    {"$set": {"city": "서울특별시"}}
)
print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")

Matched: 0, Modified: 0


In [36]:
result = users_collection.find({"city":"서울"})
print(len(list(result)))
result = users_collection.find({"city":"서울특별시"})
print(len(list(result)))

0
3


In [43]:
cnt = users_collection.count_documents({})
print("전체 문서수=",cnt)
cnt = users_collection.count_documents(
    {"city":"서울특별시"})
print("서울특별시 문서수=",cnt)

전체 문서수= 5
서울특별시 문서수= 3


In [45]:
# '장영실'의 skills 배열에 'MongoDB' 추가
print(users_collection.find_one({"name":"장영실"}))
users_collection.update_one(
    {"name": "장영실"},
    {"$push": {"skills": "MongoDB"}}
)
print(users_collection.find_one({"name":"장영실"}))

{'_id': ObjectId('6847a0908f5735a55d32acc7'), 'name': '장영실', 'age': 28, 'city': '부산', 'skills': ['Python', 'Engineering']}
{'_id': ObjectId('6847a0908f5735a55d32acc7'), 'name': '장영실', 'age': 28, 'city': '부산', 'skills': ['Python', 'Engineering', 'MongoDB']}


In [55]:
print(f"남은 사용자 수:,{users_collection.count_documents({})}")

남은 사용자 수:,5


In [56]:
import pymongo
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['performance_db']
# 기존 컬렉션이 있다면 삭제
db.perf_users.drop()
perf_users_collection = db['perf_users']

# 약 10만 개의 샘플 데이터 생성 (시간이 조금 걸릴 수 있습니다)
if perf_users_collection.count_documents({}) == 0:
    print("샘플 데이터 생성 중...")
    users_to_insert = []
    for i in range(100000):
        users_to_insert.append({"user_id": i, "name": f"user_{i}", "country": "KR"})
    perf_users_collection.insert_many(users_to_insert)
    print("샘플 데이터 생성 완료!")


샘플 데이터 생성 중...
샘플 데이터 생성 완료!


In [57]:
perf_users_collection.count_documents({})

100000

In [58]:
# 'user_id' 필드에 오름차순(ASCENDING) 인덱스 생성
# pymongo.ASCENDING 오름차순 정렬
# pymongo.DESCENDING 내림차순 정렬
perf_users_collection.create_index([("user_id", pymongo.ASCENDING)])
print("'user_id' 필드에 인덱스가 생성되었습니다.")


'user_id' 필드에 인덱스가 생성되었습니다.


In [78]:
# 'name' 필드에 고유 인덱스 생성
# unique=True 중복금지 
# name=인덱스이름
perf_users_collection.create_index([("name", pymongo.ASCENDING)], 
                                   unique=True, 
                                   name="perf_users_collection.unique.name") # 이름 지정
print("'name' 필드에 고유 인덱스가 생성되었습니다.")

# 중복 데이터 삽입 시도 -> 에러 발생!
try:
    perf_users_collection.insert_one({"name": "user_1", "country": "US"})
except pymongo.errors.DuplicateKeyError as e:
    print(f"고유 인덱스 위반! -> {e}")


'name' 필드에 고유 인덱스가 생성되었습니다.
고유 인덱스 위반! -> E11000 duplicate key error collection: performance_db.perf_users index: perf_users_collection.unique.name dup key: { name: "user_1" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: performance_db.perf_users index: perf_users_collection.unique.name dup key: { name: "user_1" }', 'keyPattern': {'name': 1}, 'keyValue': {'name': 'user_1'}}


In [61]:
for u in perf_users_collection.find({}).limit(5):
    print(u)

{'_id': ObjectId('6847cb198f5735a55d32acc9'), 'user_id': 0, 'name': 'user_0', 'country': 'KR'}
{'_id': ObjectId('6847cb198f5735a55d32acca'), 'user_id': 1, 'name': 'user_1', 'country': 'KR'}
{'_id': ObjectId('6847cb198f5735a55d32accb'), 'user_id': 2, 'name': 'user_2', 'country': 'KR'}
{'_id': ObjectId('6847cb198f5735a55d32accc'), 'user_id': 3, 'name': 'user_3', 'country': 'KR'}
{'_id': ObjectId('6847cb198f5735a55d32accd'), 'user_id': 4, 'name': 'user_4', 'country': 'KR'}


In [67]:
# country를 먼저, 그 다음 user_id 순서로 복합 인덱스 생성
# 'country'가 'KR'이고 'user_id'가 50000 이상인 사용자를 찾는 쿼리에 최적화됨
perf_users_collection.create_index([
    ("country", pymongo.ASCENDING),
    ("user_id", pymongo.DESCENDING) # 방향은 쿼리에 맞춰 설정 가능
])
print("복합 인덱스(country, user_id)가 생성되었습니다.")


OperationFailure: An existing index has the same name as the requested index. When index names are not specified, they are auto generated and can cause conflicts. Please refer to our documentation. Requested index: { v: 2, unique: true, key: { country: 1, user_id: -1 }, name: "country_1_user_id_-1" }, existing index: { v: 2, key: { country: 1, user_id: -1 }, name: "country_1_user_id_-1" }, full error: {'ok': 0.0, 'errmsg': 'An existing index has the same name as the requested index. When index names are not specified, they are auto generated and can cause conflicts. Please refer to our documentation. Requested index: { v: 2, unique: true, key: { country: 1, user_id: -1 }, name: "country_1_user_id_-1" }, existing index: { v: 2, key: { country: 1, user_id: -1 }, name: "country_1_user_id_-1" }', 'code': 86, 'codeName': 'IndexKeySpecsConflict'}

In [111]:
# 현재 컬렉션에 생성된 모든 인덱스 확인
for index in perf_users_collection.list_indexes():
    print(index)

# 특정 인덱스 삭제
# perf_users_collection.drop_index("name_1") # 인덱스 이름으로 삭제

# 모든 인덱스 삭제 (_id 인덱스 제외)
# perf_users_collection.drop_indexes()


SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('name', 1)])), ('name', 'perf_users_collection.unique.name'), ('unique', True)])


In [80]:
db = client['aggregation_db']
db.inventory.drop()
inventory_collection = db.inventory
inventory_collection.insert_many([
   {"item": "journal", "qty": 25, "size": { "h": 14, "w": 21, "uom": "cm" }, "status": "A"},
   {"item": "notebook", "qty": 50, "size": { "h": 8.5, "w": 11, "uom": "in" }, "status": "A"},
   {"item": "paper", "qty": 100, "size": { "h": 8.5, "w": 11, "uom": "in" }, "status": "D"},
   {"item": "planner", "qty": 75, "size": { "h": 22.85, "w": 30, "uom": "cm" }, "status": "D"},
   {"item": "postcard", "qty": 45, "size": { "h": 10, "w": 15.25, "uom": "cm" }, "status": "A"}
])
print("재고 데이터가 준비되었습니다.")


재고 데이터가 준비되었습니다.


In [81]:
# status가 "A"인 문서들만 필터링
pipeline = [
    {"$match": {"status": "A"}}
]
results = inventory_collection.aggregate(pipeline)
for doc in results:
    print(doc)


{'_id': ObjectId('6847d48e8f5735a55d34336b'), 'item': 'journal', 'qty': 25, 'size': {'h': 14, 'w': 21, 'uom': 'cm'}, 'status': 'A'}
{'_id': ObjectId('6847d48e8f5735a55d34336c'), 'item': 'notebook', 'qty': 50, 'size': {'h': 8.5, 'w': 11, 'uom': 'in'}, 'status': 'A'}
{'_id': ObjectId('6847d48e8f5735a55d34336f'), 'item': 'postcard', 'qty': 45, 'size': {'h': 10, 'w': 15.25, 'uom': 'cm'}, 'status': 'A'}


In [82]:
# 사이즈 단위(uom)별로 그룹화하여, 각 그룹의 아이템 개수와 총 수량 계산
pipeline = [
    {
        "$group": {
            "_id": "$size.uom",  # $를 붙여 필드 값을 참조
            "item_count": {"$sum": 1},  # 각 문서마다 1씩 더해 개수를 셈
            "total_quantity": {"$sum": "$qty"} # 그룹내 문서내 qty필드의 합
        }
    }
]
results = inventory_collection.aggregate(pipeline)
for doc in results:
    print(doc)
# {'_id': 'in', 'item_count': 2, 'total_quantity': 150}
# {'_id': 'cm', 'item_count': 3, 'total_quantity': 145}


{'_id': 'cm', 'item_count': 3, 'total_quantity': 145}
{'_id': 'in', 'item_count': 2, 'total_quantity': 150}


In [83]:
# item 이름은 'product_name'으로, qty는 그대로, _id는 제외하여 출력
pipeline = [
    {
        "$project": {
            "_id": 0,
            "product_name": "$item",
            "qty": 1  # 1을 주면 해당 필드를 포함
        }
    }
]
results = inventory_collection.aggregate(pipeline)
for doc in results:
    print(doc)


{'qty': 25, 'product_name': 'journal'}
{'qty': 50, 'product_name': 'notebook'}
{'qty': 100, 'product_name': 'paper'}
{'qty': 75, 'product_name': 'planner'}
{'qty': 45, 'product_name': 'postcard'}


In [87]:
# status가 'A'인 상품만 골라서, 사이즈 단위별로 총 수량을 계산하고, 총 수량이 많은 순으로 정렬
pipeline = [
    {"$match": {"status": "A"}},
    {
        "$group": {
            "_id": "$size.uom",
            "total_quantity": {"$sum": "$qty"}
        }
    },
    {"$sort": {"total_quantity": -1}} # -1은 내림차순
]
results = inventory_collection.aggregate(pipeline)
print("--- Aggregation 종합 예제 ---")
for doc in results:
    print(doc)


--- Aggregation 종합 예제 ---
{'_id': 'cm', 'total_quantity': 70}
{'_id': 'in', 'total_quantity': 50}


In [121]:
from pymongo import MongoClient

client = MongoClient()
db = client.testdb
myusers = db.myusers
myusers.drop()  # 기존 데이터 초기화

myusers.insert_many([
    { "name": "Alice", "age": 25, "gender": "female", "hobbies": ["reading", "yoga"] },
    { "name": "Bob", "age": 30, "gender": "male", "hobbies": ["gaming", "soccer"] },
    { "name": "Charlie", "age": 35, "gender": "male", "hobbies": ["fishing", "reading"] },
    { "name": "Diana", "age": 28, "gender": "female", "hobbies": ["baking", "traveling"] },
    { "name": "Evan", "age": 40, "gender": "male", "hobbies": ["traveling"] }
])


InsertManyResult([ObjectId('6847e3768f5735a55d3433aa'), ObjectId('6847e3768f5735a55d3433ab'), ObjectId('6847e3768f5735a55d3433ac'), ObjectId('6847e3768f5735a55d3433ad'), ObjectId('6847e3768f5735a55d3433ae')], acknowledged=True)

In [122]:
# 문제: myusers 컬렉션의 모든 문서를 조회하고 출력하시오.
for doc in myusers.find():
    print(doc)

{'_id': ObjectId('6847e3768f5735a55d3433aa'), 'name': 'Alice', 'age': 25, 'gender': 'female', 'hobbies': ['reading', 'yoga']}
{'_id': ObjectId('6847e3768f5735a55d3433ab'), 'name': 'Bob', 'age': 30, 'gender': 'male', 'hobbies': ['gaming', 'soccer']}
{'_id': ObjectId('6847e3768f5735a55d3433ac'), 'name': 'Charlie', 'age': 35, 'gender': 'male', 'hobbies': ['fishing', 'reading']}
{'_id': ObjectId('6847e3768f5735a55d3433ad'), 'name': 'Diana', 'age': 28, 'gender': 'female', 'hobbies': ['baking', 'traveling']}
{'_id': ObjectId('6847e3768f5735a55d3433ae'), 'name': 'Evan', 'age': 40, 'gender': 'male', 'hobbies': ['traveling']}


In [135]:
# 문제: age가 30 이상인 사용자의 name과 age를 출력하시오.
for users in myusers.find({"age":{"$gte":30}}):
    print(users)

{'_id': ObjectId('6847e3768f5735a55d3433ab'), 'name': 'Bob', 'age': 30, 'gender': 'male', 'hobbies': ['gaming', 'soccer']}
{'_id': ObjectId('6847e3768f5735a55d3433ac'), 'name': 'Charlie', 'age': 35, 'gender': 'male', 'hobbies': ['fishing', 'reading']}
{'_id': ObjectId('6847e3768f5735a55d3433ae'), 'name': 'Evan', 'age': 40, 'gender': 'male', 'hobbies': ['traveling']}


In [137]:
# 문제: gender가 'female'인 사용자의 수를 출력하시오.
for f in myusers.find({"gender":"female"}):
    print(f)

{'_id': ObjectId('6847e3768f5735a55d3433aa'), 'name': 'Alice', 'age': 25, 'gender': 'female', 'hobbies': ['reading', 'yoga']}
{'_id': ObjectId('6847e3768f5735a55d3433ad'), 'name': 'Diana', 'age': 28, 'gender': 'female', 'hobbies': ['baking', 'traveling']}


In [138]:
# 문제: hobbies 배열에 'reading'이 포함된 사용자를 출력하시오.
for h in myusers.find({"hobbies":"reading"}):
    print(h)

{'_id': ObjectId('6847e3768f5735a55d3433aa'), 'name': 'Alice', 'age': 25, 'gender': 'female', 'hobbies': ['reading', 'yoga']}
{'_id': ObjectId('6847e3768f5735a55d3433ac'), 'name': 'Charlie', 'age': 35, 'gender': 'male', 'hobbies': ['fishing', 'reading']}


In [140]:
# 문제: 나이(age)를 기준으로 오름차순 정렬하고 상위 3명을 출력하시오.
for a in myusers.find().sort("age", 1).limit(3):
    print(a)

{'_id': ObjectId('6847e3768f5735a55d3433aa'), 'name': 'Alice', 'age': 25, 'gender': 'female', 'hobbies': ['reading', 'yoga']}
{'_id': ObjectId('6847e3768f5735a55d3433ad'), 'name': 'Diana', 'age': 28, 'gender': 'female', 'hobbies': ['baking', 'traveling']}
{'_id': ObjectId('6847e3768f5735a55d3433ab'), 'name': 'Bob', 'age': 30, 'gender': 'male', 'hobbies': ['gaming', 'soccer']}


In [142]:
# 문제: hobbies에 'traveling'이 있는 사용자들의 이름만 출력하시오.
for n in myusers.find({"hobbies":"traveling"}, {"name":1}):
    print(n)

{'_id': ObjectId('6847e3768f5735a55d3433ad'), 'name': 'Diana'}
{'_id': ObjectId('6847e3768f5735a55d3433ae'), 'name': 'Evan'}


In [ ]:
# 문제: gender별로 사용자 수를 집계하시오. (aggregation 사용)
